In [2]:
import pandas as pd
import numpy as np
import argparse
import hashlib
import re

In [121]:
SOURCE_CSV = 'https://shelterdata.s3.amazonaws.com/shelter1000.csv'

In [122]:
def sort_columns(data):
    data_copy = data.copy()
    format = "%m/%d/%Y %H:%M:%S %p"
    data_copy['DateTime'] = pd.to_datetime(data_copy['DateTime'], format=format)
    data_copy = data_copy.sort_values(by=['DateTime'], ascending=False).reset_index(drop=True)
    return data_copy

def split_columns(data):
    data_copy = data.copy()
    data_copy[['Neutered', 'Sex']] = data_copy['Sex upon Outcome'].str.split(expand=True)
    data_copy['Year'] = data_copy['DateTime'].dt.year
    data_copy['Month'] = data_copy['DateTime'].dt.month
    data_copy['Day'] = data_copy['DateTime'].dt.day
    return data_copy

def clean_columns(data):
    data_copy = data.copy()
    data_copy = data_copy.apply(lambda col: col.str.lower() if col.dtype == "object" else col)
    data_copy['Name'] = data_copy['Name'].apply(lambda x: re.sub(r'\*', '', str(x)))
    data_copy['Name'] = data_copy['Name'].apply(lambda x: re.sub(r'a\d{6}', 'unknown', str(x)))
    data_copy['Name'] = data_copy['Name'].apply(lambda x: re.sub(r'\d+ grams', 'unknown', str(x)))
    data_copy['Name'] = data_copy['Name'].apply(lambda x: re.sub(r'\d+g', '', str(x)))
    data_copy['Name'] = data_copy['Name'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))
    data_copy['Name'] = data_copy['Name'].apply(lambda x: re.sub(r"^ +| +$", "", str(x)))
    data_copy['Name'] = data_copy['Name'].apply(lambda x: re.sub(r'^\s*$', "unknown", str(x)))
    data_copy['Name'] = data_copy['Name'].replace('nan', 'unknown')
    data_copy['Sex'] = data_copy['Sex'].replace(np.nan, 'unknown')
    data_copy['Outcome Subtype'] = data_copy['Outcome Subtype'].replace(np.nan, 'none')
    data_copy['Neutered'] = data_copy['Neutered'].replace('intact', 'no')
    data_copy['Neutered'] = data_copy['Neutered'].replace(['neutered', 'spayed'], 'yes')
    return data_copy

def drop_columns(data):
    columns = ['MonthYear', 'Age upon Outcome', 'Sex upon Outcome']
    data_copy = data.drop(columns, axis=1, inplace=False)
    return data_copy

def rename_columns(data):
    data_copy = data.copy()
    data_copy.columns = ['animal_natural_key', 'animal_name', 'outcome_date', 'animal_dob', 'outcome_type', 
                         'outcome_type_subtype', 'animal_type', 'animal_breed', 'animal_color', 
                         'outcome_type_neutered', 'animal_sex', 'outcome_date_year', 'outcome_date_month', 
                         'outcome_date_day']
    return data_copy

def prep_data(source_csv):
    data_copy = pd.read_csv(source_csv)
    data_copy = sort_columns(data_copy)
    data_copy = split_columns(data_copy)
    data_copy = clean_columns(data_copy)
    data_copy = drop_columns(data_copy)
    data_copy = rename_columns(data_copy)
    return data_copy

In [123]:
prepped_data = prep_data(SOURCE_CSV)

In [124]:
def transform_animal_dim(data):
    data_copy = data[['animal_natural_key', 'animal_name', 'animal_dob', 'animal_type',
                      'animal_breed', 'animal_color', 'animal_sex']].copy()
    surrogate_keys = [hashlib.md5(row.astype(str).str.cat(sep='').encode('utf-8')).hexdigest() for _, row in data_copy.iterrows()]
    data_copy['animal_id'] = surrogate_keys
    data_copy.drop_duplicates(inplace=True)
    data_copy.reset_index(drop=True, inplace=True)
    return surrogate_keys, data_copy    

def transform_date_dim(data):
    data_copy = data[['outcome_date', 'outcome_date_year', 'outcome_date_month', 'outcome_date_day']].copy()
    data_copy['outcome_date'] = data_copy['outcome_date'].dt.date
    surrogate_keys = [hashlib.md5(row.astype(str).str.cat(sep='').encode('utf-8')).hexdigest() for _, row in data_copy.iterrows()]
    data_copy['outcome_date_id'] = surrogate_keys
    data_copy.drop_duplicates(inplace=True)
    data_copy.reset_index(drop=True, inplace=True)
    return surrogate_keys, data_copy

def transform_outcome_dim(data):
    data_copy = data[['outcome_type', 'outcome_type_subtype', 'outcome_type_neutered']].copy()
    surrogate_keys = [hashlib.md5(row.astype(str).str.cat(sep='').encode('utf-8')).hexdigest() for _, row in data_copy.iterrows()]
    data_copy['outcome_date_id'] = surrogate_keys
    data_copy.drop_duplicates(inplace=True)
    data_copy.reset_index(drop=True, inplace=True)
    return surrogate_keys, data_copy

def transform_fact_table(data, animal_keys, date_keys, outcome_keys):
    data_copy = data[['animal_natural_key']].copy()
    data_copy['animal_id'] = animal_keys
    data_copy['outcome_date_id'] = date_keys
    data_copy['outcome_type_id'] = outcome_keys
    return data_copy

def tranform_data(source_csv):
    data = prep_data(source_csv)
    animal_keys, animal_dim = transform_animal_dim(data)
    date_keys, date_dim = transform_date_dim(data)
    outcome_keys, outcome_dim = transform_outcome_dim(data)
    fact_table = transform_fact_table(data, animal_keys, date_keys, outcome_keys)
    return animal_dim, date_dim, outcome_dim, fact_table

In [125]:
animal_dim, date_dim, outcome_dim, fact_table = tranform_data(SOURCE_CSV)

In [126]:
animal_dim

,animal_natural_key,animal_name,animal_dob,animal_type,animal_breed,animal_color,animal_sex,animal_id
0,a882145,javier,04/16/2023,cat,domestic shorthair,brown tabby,male,858eac5c5c26f77bc549b118cf608abd
1,a879902,fluff,03/26/2023,cat,domestic shorthair,black,female,21e1f41948cfccb7a34c3c3a25b658ee
2,a873253,peach cobbler,07/23/2020,cat,domestic medium hair,black,female,4d176a54ca0fafadc47f2097559627a4
3,a876652,gravy,03/17/2023,cat,domestic shorthair,blue tabby,male,61d12eed42cb6e963718c1df41b1ab67
4,a880165,unknown,05/05/2021,cat,domestic shorthair,orange tabby,male,ee90f154b5d9cc78533780f51686ada9
...,...,...,...,...,...,...,...,...
992,a664996,unknown,10/05/2013,dog,australian cattle dog mix,tricolor,female,017b709796dd727b953aa5375ca10d23
993,a664661,lily,08/07/2013,cat,domestic shorthair mix,black/white,female,f68ea6838dad3c9cdf9ef8ff579ec617
994,a657197,gecko,05/12/2013,cat,domestic shorthair mix,brown tabby,male,281c2698ca896333154c0b993522a16e
995,a662741,todd,08/11/2013,cat,domestic shorthair mix,black,male,c81202a5bb658e94a1444884014806e8


In [127]:
date_dim

,outcome_date,outcome_date_year,outcome_date_month,outcome_date_day,outcome_date_id
0,2023-08-09,2023,8,9,e868a056913d31b88aa617b85fa647ee
1,2023-05-28,2023,5,28,9c17ed2ce8da701cad89a0ae0439ab12
2,2023-05-12,2023,5,12,c78a198f679d493db4fb7c84491a0024
3,2023-05-08,2023,5,8,cffd23b1522dc992560c5975353b1f92
4,2023-04-30,2023,4,30,2c18754b84525bd31c0651b674f9a40f
...,...,...,...,...,...
841,2013-10-13,2013,10,13,b1f669b26d589051c31cf9d12b40ef86
842,2013-10-12,2013,10,12,6bbb031babc07660a32220417325cd4c
843,2013-10-09,2013,10,9,4070f835ed741fcefa8e9114d2c05c4e
844,2013-10-05,2013,10,5,8f7c06a92b0c0f084dc05963f7081e84


In [128]:
outcome_dim

,outcome_type,outcome_type_subtype,outcome_type_neutered,outcome_date_id
0,adoption,foster,yes,f545dfe47822060f410261b2fc0e5627
1,adoption,none,yes,f3b13c8275bc62e819c6fa9875676ac9
2,transfer,snr,no,95322443e7081ba69dd2ab5d105cdc80
3,transfer,partner,no,092d2cedb9a8e6814813e62f2edfdd8c
4,died,in kennel,unknown,330b21ca186567347d0568830517c104
5,euthanasia,suffering,no,ee8e6a873d7b4399c9188c0d565cb308
6,adoption,foster,no,8f6a375e703a856ee6006690a9822b21
7,return to owner,none,no,1ca4b78972674e3900b5461252d4873b
8,adoption,none,unknown,cd14131545125e40109ef5691256544a
9,euthanasia,rabies risk,unknown,ef3266741d2fdd203fdbbef3c6e4e8e1


In [129]:
fact_table

,animal_natural_key,animal_id,outcome_date_id,outcome_type_id
0,a882145,858eac5c5c26f77bc549b118cf608abd,e868a056913d31b88aa617b85fa647ee,f545dfe47822060f410261b2fc0e5627
1,a879902,21e1f41948cfccb7a34c3c3a25b658ee,9c17ed2ce8da701cad89a0ae0439ab12,f545dfe47822060f410261b2fc0e5627
2,a873253,4d176a54ca0fafadc47f2097559627a4,9c17ed2ce8da701cad89a0ae0439ab12,f3b13c8275bc62e819c6fa9875676ac9
3,a876652,61d12eed42cb6e963718c1df41b1ab67,c78a198f679d493db4fb7c84491a0024,f3b13c8275bc62e819c6fa9875676ac9
4,a880165,ee90f154b5d9cc78533780f51686ada9,cffd23b1522dc992560c5975353b1f92,95322443e7081ba69dd2ab5d105cdc80
...,...,...,...,...
995,a664661,f68ea6838dad3c9cdf9ef8ff579ec617,b1f669b26d589051c31cf9d12b40ef86,f3b13c8275bc62e819c6fa9875676ac9
996,a664941,948648898aec63b89b95010014aac945,6bbb031babc07660a32220417325cd4c,e0c0c3766c57e6e17ce67e0019fdacd0
997,a657197,281c2698ca896333154c0b993522a16e,4070f835ed741fcefa8e9114d2c05c4e,f3b13c8275bc62e819c6fa9875676ac9
998,a662741,c81202a5bb658e94a1444884014806e8,8f7c06a92b0c0f084dc05963f7081e84,092d2cedb9a8e6814813e62f2edfdd8c
